# Editor Agent

This notebook illustrates how the very first version of this agent was developed


In [1]:
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

from langgraph.graph import START, END, StateGraph
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Load environment variables
env_path = project_root / '.env'
load_dotenv(dotenv_path=env_path)

print("✓ Imports successful")
print(f"✓ Loading .env from: {env_path}")

✓ Imports successful
✓ Loading .env from: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.env


In [9]:
slug = "socotra_inside_the_alien_island"

## Prompts

In [3]:
# ============= FROZEN PROMPTS =============

magazine_profile = """ Agentic Newsroom is a magazine of discovery and wonder.
It explores the hidden corners of the planet and the universe, cutting-edge scientific mysteries, strange natural phenomena, ancient civilization, unusual places, and the latest in space exploration.
Our editorial voice blends scientific rigor, vivid storytelling, a touch of wonder, and strong narrative.
Articles are deeply researched, visually descriptive, and rich in detail.
We aim to make complex ideas accessible, without dumbing them down and to evoke awe, curiosity, and sense of scale.
"""

magazine_guardrails = """
Agentic Newsroom adheres to the following editorial standards in all published writing:

1. Agentic Newsroom does not produce or publish any form of hate speech, harassment, racism, sexism, or discriminatory content.
2. Articles may occasionally discuss pseudoscience, fringe theories, myths, or unverified claims when relevant to cultural or historical context. 
   However, Agentic Newsroom does not endorse such claims and always presents them with clear scientific, historical, or factual framing.
3. All information must be presented accurately, responsibly, and with respect for both scientific consensus and ethical journalism.
4. When uncertainty exists, Agentic Newsroom clearly distinguishes between established fact, informed interpretation, and speculation.
"""

editor_profile = """
Your name is Anthony. You are the line editor for Agentic Newsroom.
You have more than twenty years of experience editing major science and geography magazines. 
"""

editor_write_prompt = """
You are a newsroom editor whose responsibility is to produce the final article from the draft package created by the reporter.

<Magazine Profile>
{magazine_profile}
</Magazine Profile>

<Editor Profile>
{editor_profile}
</Editor Profile>

<Magazine Guardrails>
Strictly adhere to the following editorial standards:
{magazine_guardrails}
</Magazine Guardrails>

<Task>
You will receive a draft package created by the reporter. Your job is to refine this draft into its final publishable form.
</Task>

<Attribution Rules>
The reporter includes attributions for all major facts. 
As the editor, you must:

- **Preserve all meaningful, non-trivial attributions** (e.g., those referring to peer-reviewed studies, scientific analyses, UAV surveys, remote-sensing studies, conservation reports, historical accounts, IntechOpen syntheses, PMC articles, etc.).
- **Polish the attribution language** for readability, but do NOT remove or anonymize these sources.
- **Remove trivial or generic attribution** such as references to Wikipedia or other general-purpose summaries.

Your role is to maintain the factual provenance and transparency of the reporter's research while elevating the clarity and flow of the writing.
</Attribution Rules>

<Instructions>
- Your role is to **polish**, not rewrite. Preserve the reporter's structure, attribution placement, narrative flow, and all substantive content.
- Improve clarity, coherence, pacing, and readability without altering meaning or introducing new unsupported claims.
- Fix grammar, syntax, factual inconsistencies, and stylistic issues while maintaining the reporter's intended voice.
- Ensure the final article aligns with the magazine guardrails and the magazine's editorial tone.
- Do NOT remove attributions except for trivial ones (e.g., Wikipedia references). 
- Do NOT reduce attribution to vague phrases ("scientists say…") if the draft names a specific study, synthesis, or research team.
</Instructions>

<Output Format>
- Write the final article in markdown format with correct headings, subheadings, and formatting.
- Write **only** the final article. 
- No commentary, explanations, or extra text of any kind.
</Output Format>
"""

## States and Structured Outputs

In [4]:
# ============= FROZEN SCHEMAS =============

from typing import List, Optional, TypedDict
from pydantic import BaseModel, Field

class DraftPackage(BaseModel):
    """Draft package created by the Reporter."""
    full_draft: str = Field(..., description="The complete article text")
    sources: List[str] = Field(..., description="List of sources")
    flagged_facts: List[str] = Field(..., description="Facts/claims that need verification")
    structural_notes: List[str] = Field(..., description="Notes about the draft structure")

class FinalArticle(BaseModel):
    final_article: str = Field(..., description="The final article text")

class EditorState(TypedDict):
    """State for the Editor subgraph."""
    draft_package: DraftPackage
    final_article: Optional[FinalArticle]

## Tools and Configuration

In [5]:
model = init_chat_model(model="openai:gpt-5.1", reasoning_effort="medium")
# model = init_chat_model(model="openai:gpt-5-mini", reasoning_effort="minimal")

print("✓ Configuration complete")

✓ Configuration complete


## Nodes

In [6]:
def write_final_article(state: EditorState):

    draft_package_str =  state["draft_package"].json()

    system_msg_content = editor_write_prompt.format(
        magazine_profile=magazine_profile,
        magazine_guardrails=magazine_guardrails,
        editor_profile=editor_profile
    )

    # Pass research as a HumanMessage context
    messages = [
        SystemMessage(content=system_msg_content),
        HumanMessage(content=f"Here is the draft package for you to edit:\n\n{draft_package_str}")
    ]

    structured_model = model.with_structured_output(FinalArticle)
    response = structured_model.invoke(messages)
    
    return {
        "final_article": response
    }


### Graph

In [10]:
builder = StateGraph(EditorState)
builder.add_node("write_final_article", write_final_article)
builder.add_edge(START, "write_final_article")
builder.add_edge("write_final_article", END)

editor_agent = builder.compile()

print("✓ Graph compiled")
print("\nGraph structure (ASCII):")
print(editor_agent.get_graph().draw_ascii())

✓ Graph compiled

Graph structure (ASCII):
     +-----------+       
     | __start__ |       
     +-----------+       
            *            
            *            
            *            
+---------------------+  
| write_final_article |  
+---------------------+  
            *            
            *            
            *            
      +---------+        
      | __end__ |        
      +---------+        


## Test Run

In [11]:
def load_draft_package(slug: str) -> DraftPackage:
    """Load a DraftPackage from the tmp directory."""
    path = project_root / "tmp" / slug / "draft_package.json"
    if not path.exists():
        raise FileNotFoundError(f"DraftPackage not found at {path}")
    
    with open(path, "r") as f:
        data = json.load(f)
    return DraftPackage(**data)

# Load the draft package
draft_package = load_draft_package(slug)

print("✓ Draft Package Loaded")
print(f"  Draft Length: {len(draft_package.full_draft)} characters")
print(f"  Sources: {len(draft_package.sources)}")
print(f"  Flagged Facts: {len(draft_package.flagged_facts)}")

✓ Draft Package Loaded
  Draft Length: 8796 characters
  Sources: 5
  Flagged Facts: 3


In [12]:
result = editor_agent.invoke({
    "draft_package": draft_package
})

print("✓ Editor agent execution complete")

/Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.venv/lib/python3.12/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)
/var/folders/nx/3q5slrkn0_q055rpt3whrsbm0000gn/T/ipykernel_26214/1363391115.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  draft_package_str =  state["draft_package"].json()


✓ Editor agent execution complete


## Display Final Article

In [13]:
from IPython.display import display, Markdown

if "final_article" in result and result["final_article"]:
    final_article = result["final_article"]
    
    print(f"✅ Final Article Generated")
    print(f"   Length: {len(final_article.final_article)} characters")
    print("\n" + "="*80 + "\n")
    
    display(Markdown(final_article.final_article))
else:
    print("⚠️ No final article generated")

✅ Final Article Generated
   Length: 8261 characters




On the limestone plateau of Firmihin, the trees look like something sketched for a science‑fiction set and then mistakenly left on Earth. Their trunks rise straight and pale before exploding into tight, green umbrellas that cast perfectly circular shadows on the rock. When one is cut or wounded, a dark sap oozes from the bark and dries to the color of old blood.

This is *Dracaena cinnabari*, the dragon’s blood tree—Socotra’s emblem and, to many visitors, proof that this island in the northwest Indian Ocean really does resemble another planet. According to a 2018 paper archived in the U.S. National Library of Medicine’s PMC database, the species is found only here and only between about 300 and 1,480 meters above sea level, tracking the belt of fog and higher moisture that clings to Socotra’s mountains and plateaus.

Socotra is a Yemeni island, but on the map it hangs alone, nearly 380 kilometers south of the Arabian Peninsula and about 232 kilometers east of the Horn of Africa. Roughly 132 kilometers long and 42 kilometers wide, it makes up about 95 percent of the land in the Socotra Archipelago. The rest of the governorate is scattered in smaller pieces: Abd al Kuri, Samhah, Darsah, and a handful of rocky islets described in Yemen’s Socotra Governorate records.

## A continent shard adrift

Socotra’s otherworldly feel begins with its geology. The island is not volcanic, like Hawaii or the Galápagos, but a continental fragment—a sliver of ancient Gondwana that broke away from Africa. Geologists link its separation to Miocene‑age rifting that opened the Gulf of Aden, leaving Socotra as an isolated block of continental crust marooned in tropical seas.

That deep history set the stage for an evolutionary experiment. The island is built largely of limestone, lifted and carved into highlands by the Hajhir Mountains, where granite pokes through the carbonate cap. The lowlands are starkly arid, receiving around 150 millimeters of rainfall a year. Most of that comes with the northeast monsoon from November to March; from April to October the southwest monsoon brings fierce, dry winds.

Higher up, the pattern flips. Moist air off the sea condenses into fog, wrung out as it rises over the mountains. This orographic mist can deliver up to 1,000 millimeters of moisture per year to the highlands, feeding narrow, perennial streams found only in the mountains. To a drought‑adapted tree like *D. cinnabari*, that band of cloud and drizzle is life itself.

## Where evolution went off‑script

Within that patchwork of foggy summits and sun‑blasted plains, evolution had time—millions of years of it—to improvise. Botanical surveys of the island estimate that roughly one‑third of Socotra’s plant species occur nowhere else on Earth. A 2023 paper in the journal *Forests* refined that figure for vascular plants to about 37 percent, underscoring just how unusual Socotra’s flora has become.

Dragon’s blood trees are only the most famous expression of that isolation. The 2018 PMC article and subsequent *Forests* studies describe *D. cinnabari* as one of several arborescent, or tree‑forming, members of the *Dracaena* group, many of which are insular and threatened across their ranges. On Socotra, one 2022 *Forests* study, using field data and habitat modeling to locate suitable zones for conservation and restoration, estimated a population of around 80,000 dragon’s blood trees.

Those studies emphasize that the trees’ current distribution hugs the foggy mid‑ and high‑elevation zones, making them especially sensitive to shifts in cloud patterns or land use. To understand how these strange forests are changing, a team led by Czech researchers inventoried dragon’s blood trees on the Firmihin plateau between 2010 and 2011, as summarized in the 2023 *Forests* article. Working across roughly 700 hectares, they laid out more than a hundred circular plots, measured every tree taller than 1.3 meters, and mapped the distinctive multi‑branched crowns—establishing a baseline for future monitoring.

Botanists and biogeographers see the island as a natural laboratory: a continental shard that carried an ancient African flora out to sea, then left it to adapt to an extreme, wind‑scoured climate. The island’s Tertiary‑age separation from the mainland, noted in the PMC and *Forests* papers, helps explain why so many lineages on Socotra have diverged into their own species.

## An island of scripts and stories

Socotra may look alien, but it has been woven into human history for thousands of years. Archaeological work near the modern town of Hadibo has revealed an Oldowan‑style stone tool culture, hinting at very ancient human presence. Later, the island appears in the *Periplus of the Erythraean Sea*, a Greek‑language guide to Indian Ocean trade from around the first century.

Inside Hoq Cave on Socotra’s north coast, explorers have documented inscriptions in multiple scripts, from South Arabian to Greek and Indian Brahmi, dating between the first century BCE and the sixth century CE. Together, they suggest that sailors from around the region once sheltered under the island’s cliffs, carving their names and prayers into the cave walls while waiting for the monsoon winds to turn.

Today, the people who call the island home are known as Socotrans. Most speak both Arabic and Soqotri, a Modern South Arabian language that, like the island’s flora, evolved in relative isolation.

Politically, Socotra has spent centuries tethered to distant powers. Historical records trace its passage through the Mahra Sultanate, a brief Portuguese occupation in the early 1500s, and later British control under the Bombay Presidency and then the Aden Protectorate. After South Yemen’s independence in 1967, Socotra was folded into Aden Governorate, and then Hadhramaut, before being designated its own governorate in 2013.

That administrative shuffling has continued in the twenty‑first century, often reflecting wider turmoil in Yemen. In April 2018, the United Arab Emirates deployed troops to Socotra and took administrative control of the island’s airport and seaport. Saudi forces arrived in May 2018 and brokered an agreement that restored formal control to Yemen’s government. In 2020, however, the UAE‑backed Southern Transitional Council, a secessionist group, seized effective control of the archipelago—a situation that, by most recent accounts, continues today.

## A fragile World Heritage

Socotra’s singular mix of geology, climate, and life did not go unnoticed by conservation bodies. The island was recognized as a UNESCO biosphere reserve in 2003, according to the 2023 *Forests* article, and inscribed as a World Heritage Site in 2008. Those designations acknowledge not only the dragon’s blood forests but the full suite of endemic plants and the broader island landscape.

Scientists working on Socotra increasingly frame the archipelago as a test case for how isolated ecosystems might weather the twenty‑first century. Because dragon’s blood trees and many other endemic species are tightly linked to fog‑fed mountain habitats, any change in monsoon rhythms or cloud formation could redraw the map of where they can survive. The 2022 *Forests* modeling study, for example, sought to identify pockets of land where restoration would be most successful under changing conditions, while also pointing out that dragon tree species globally tend to be rare and endangered, often with small, insular populations.

At the same time, Socotra’s people are navigating the pressures of conflict, isolation, and the outside world’s growing fascination with their home. Reliable data on tourism and its impacts are scarce in the sources synthesized here, but conservationists and local authorities will likely face a familiar dilemma seen in other remote World Heritage Sites: how to balance economic opportunities from visitors with the need to protect fragile habitats and respect local culture.

For now, the dragon’s blood trees still stand on their wind‑carved plateaus, catching the fog in their umbrella crowns. Their red resin once fed ancient trade routes and medicinal traditions; today, their survival may tell us how well an “alien” ecosystem can endure on an increasingly crowded and warming planet.

In [14]:
def save_final_article(final_article: FinalArticle, slug: str):
    """
    Save a FinalArticle to the slug directory.
    
    Saves both:
    - final_article.json (can be loaded back into FinalArticle object)
    - final_article.md (human-readable markdown version)
    
    Args:
        final_article: The FinalArticle to save
        slug: The article slug (directory name in tmp/)
    
    Returns:
        Path to the article directory
    """
    # Define paths
    tmp_dir = project_root / "tmp"
    article_dir = tmp_dir / slug
    
    # Ensure directory exists
    article_dir.mkdir(parents=True, exist_ok=True)
    
    # Save JSON (can be loaded back into FinalArticle)
    json_path = article_dir / "final_article.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        f.write(final_article.model_dump_json(indent=2))
    
    # Save Markdown version (just the article text)
    md_path = article_dir / "final_article.md"
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(final_article.final_article)
    
    print(f"✅ Final article saved to: {article_dir}")
    print(f"   - JSON: {json_path.name}")
    print(f"   - Markdown: {md_path.name}")
    
    return article_dir

# Save the final article
if "final_article" in result and result["final_article"]:
    saved_dir = save_final_article(result["final_article"], slug)
else:
    print("⚠️ No final article to save")

✅ Final article saved to: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/tmp/socotra_inside_the_alien_island
   - JSON: final_article.json
   - Markdown: final_article.md
